In [ ]:
import json
import pandas as pd
import requests

from collections import defaultdict
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import warnings

warnings.filterwarnings('ignore')

with open('user.txt', 'r') as file:
	STEAM_ID, API_KEY = file.read().split('\n')

GAMES_URL = 'https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001?key={}'
GAMES_URL = GAMES_URL.format(API_KEY)
GAMES_URL = GAMES_URL + '&include_played_free_games=1&skip_unvetted_apps=0&include_appinfo=1&steamid={}'
GAMES_URL = GAMES_URL.format(STEAM_ID)

ACHIEV_URL = '&key=' + API_KEY + '&steamid=' + STEAM_ID
ACHIEV_URL = 'https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid={}' + ACHIEV_URL

APP_URL = 'https://api.steamcmd.net/v1/info/{}'

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
response = session.get(GAMES_URL)
result = response.json()["response"]["games"]
result = sorted(result, key=lambda g: g["playtime_forever"], reverse=True)
df = pd.json_normalize(result)

### Collecting Achievements

In [ ]:
achievements_data = []

for appid, name in zip(df['appid'], df['name']):
	response = session.get(ACHIEV_URL.format(appid))

	try:
		result = response.json()['playerstats']['achievements']
		achievements = pd.json_normalize(result)
		unlocked = (achievements['achieved'] == 1).sum()
		locked = (achievements['achieved'] == 0).sum()
		total = unlocked + locked
		perfect = int(unlocked == total)

	except KeyError:
		unlocked, locked, total, perfect = 0, 0, 0, 0

	achievements_data.append([appid, unlocked, locked, total, perfect])
	print("Collected achievements for {}".format(name))

achievements_df = pd.DataFrame(achievements_data, columns=['appid', 'achievements_unlocked', 'achievements_locked', 'achievements_total', 'achievements_perfect'])
df = df.merge(achievements_df, on='appid')

### Collecting Tags

In [ ]:
with open("steamdb_tags.json", "r", encoding="utf-8") as f:
	tags_dict = json.load(f)

for appid, name in zip(df['appid'], df['name']):
	response = session.get(APP_URL.format(appid))
	try:
		tags = response.json()['data'][str(appid)]['common']['store_tags']
	except KeyError:
		print("No tags for {}".format(name))
		continue
	tags = dict(sorted(tags.items(), key=lambda x: int(x[0])))

	for n, tag in tags.items():
		tag = tags_dict[str(tag)]
		df.loc[df.appid == appid, 'tag_{}'.format(n)] = tag

	print("Collected tags for {}".format(name))

### Calcuting Total Playtime using Tags

In [ ]:
weights = [round(max(0, 1 - i * 0.0265), 2) for i in range(20)]

tag_playtime = defaultdict(float)
tag_playtime_weighted = defaultdict(float)

for _, row in df.iterrows():
	playtime = row['playtime_forever']
	for i in range(20):
		tag_column = f'tag_{i}'
		if pd.notna(row[tag_column]):
			tag = row[tag_column]
			tag_playtime[tag] += playtime
			tag_playtime_weighted[tag] += playtime * weights[i]

tag_playtime_df = pd.DataFrame(tag_playtime.items(), columns=["Tag", "Total Playtime"])
tag_playtime_df['Total Playtime (hours)'] = round(tag_playtime_df['Total Playtime'] / 60, 2)

tag_playtime_weighted_df = pd.DataFrame(tag_playtime_weighted.items(), columns=["Tag", "Total Weighted Playtime"])
tag_playtime_weighted_df['Total Weighted Playtime (hours)'] = round(tag_playtime_weighted_df['Total Weighted Playtime'] / 60, 2)
tag_playtime_weighted_df['Total Weighted Playtime'] = round(tag_playtime_weighted_df['Total Weighted Playtime'], 2)

tag_playtime_df = pd.merge(tag_playtime_df, tag_playtime_weighted_df, on="Tag")
tag_playtime_df = tag_playtime_df.sort_values(by="Total Weighted Playtime", ascending=False)